In [1]:
import torch
import numpy as np
import cv2 as cv
import torch.nn as nn
import torch.nn.functional as F

from maskrcnn_benchmark.structures.bounding_box import BoxList
from maskrcnn_benchmark.structures.boxlist_ops import boxlist_iou

In [33]:
B = 5
E = 4
embs = torch.randint(0, 3, (B, E)).float()  # [B,E]
labels = torch.randint(0, 5, (B,)).float()  # [B,]

In [34]:
embs, embs.shape

(tensor([[0., 2., 1., 2.],
         [0., 1., 1., 1.],
         [2., 0., 1., 1.],
         [1., 2., 0., 2.],
         [2., 1., 1., 1.]]),
 torch.Size([5, 4]))

In [35]:
labels, labels.shape

(tensor([4., 4., 1., 1., 0.]), torch.Size([5]))

In [37]:
labels_2d = torch.atleast_2d(labels)
mask = labels_2d == labels_2d.T
mask

tensor([[ True,  True, False, False, False],
        [ True,  True, False, False, False],
        [False, False,  True,  True, False],
        [False, False,  True,  True, False],
        [False, False, False, False,  True]])

In [ ]:
def _pairwise_distances(
    embs: torch.Tensor,
    *,
    squared: bool = False,
    eps: float = 1e-16
) -> torch.Tensor:
    pass

In [45]:
def _get_anchor_positive_triplet_mask(labels: torch.Tensor) -> torch.Tensor:
    labels_eq_mask = (labels[..., None] == labels[None, ...])  # [B,B]
    idxs_neq_mask = ~torch.eye(len(labels), dtype=torch.bool)  # [B,B]
    anchor_positive_mask = (labels_eq_mask & idxs_neq_mask)  # [B,B]
    return anchor_positive_mask

def _get_anchor_negative_triplet_mask(labels: torch.Tensor) -> torch.Tensor:
    anchor_negative_mask = (labels[..., None] != labels[None, ...])  # [B,B]
    return anchor_negative_mask

def get_triplet_mask(labels: torch.Tensor) -> torch.Tensor:
    idxs_neq_mask = ~torch.eye(len(labels), dtype=torch.bool)  # [B,B]
    idx_i_neq_j_mask = torch.unsqueeze(idxs_neq_mask, dim=2)  # [B,B,1]
    idx_i_neq_k_mask = torch.unsqueeze(idxs_neq_mask, dim=1)  # [B,1,B]
    idx_j_neq_k_mask = torch.unsqueeze(idxs_neq_mask, dim=0)  # [1,B,B]
    triplet_idxs_neq_mask = (
        idx_i_neq_j_mask & idx_i_neq_k_mask & idx_j_neq_k_mask  # [B,B,B]
    )

    labels_eq_mask = (labels[..., None] == labels[None, ...])  # [B,B]
    label_i_eq_j = torch.unsqueeze(labels_eq_mask, dim=2)  # [B,B,1]
    label_i_neq_k = ~torch.unsqueeze(labels_eq_mask, dim=1)  # [B,1,B]
    triplet_labels_valid_mask = (label_i_eq_j & label_i_neq_k)  # [B,B,B]

    triplet_mask = (
        triplet_idxs_neq_mask & triplet_labels_valid_mask  # [B,B,B]
    )

    return triplet_mask


In [42]:
n = 3
indices_neq_mask= ~torch.eye(n, dtype=torch.bool)
mask_ij = torch.unsqueeze(indices_neq_mask, dim=2)
mask_ik = torch.unsqueeze(indices_neq_mask, dim=1)
mask_jk = torch.unsqueeze(indices_neq_mask, dim=0)
mask_ij.shape, mask_ik.shape, mask_jk.shape

(torch.Size([3, 3, 1]), torch.Size([3, 1, 3]), torch.Size([1, 3, 3]))